In [ ]:
from _util import *
import _Ohio_Simulator as Ohio
import _RL.FQE as FQE_module
import _RL.FQI as FQI
import _RL.my_gym as my_gym
import _TRIPLE as TRIPLE
import _analyze as analyze
reload(TRIPLE)
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
###################################################################################################################
unique_setting = "ohio_tau_Qs"
n_gpu = 16

rep = 200
tau = 1
N = 20

tp_path = "target_policies/1003_gamma95_sd10/1003_gamma95_sd10_50"
########################################################################
sd_G = 10
T = 200
gamma = .95
init_Q_ratio = 1
FQE_paras = {"lr" : 2e-5, "batch_size" : 32, "nn_verbose" : 0, "hiddens" : 256
             , "verbose" : 1, "test_freq" : 50
             , "eps" : 0.0001, "max_iter" : 200, "init_Q_ratio" : init_Q_ratio
             , "max_depth" : 50, "n_estimators" : 1000, "min_samples_leaf" : 10
             ,  "use_RF" : 1}
################################################################################################################################################ 
###################################################### Default Settings ######################################################
################################################################################################################################################
incomplete_ratio = 5
hyperparameters = [sd_G, N, T, gamma, tau, init_Q_ratio, incomplete_ratio]
################## estimate the target policy [fixed number of iterations?] ##################
setting = EST()[7:9] + EST()[10:13]+ "_"  + unique_setting + "_" + "N" + str(N) + "_" + "T" + str(T) + "_" + "sd" + str(sd_G) + "_" + "initQ" + str(init_Q_ratio) + "_"  + "tau" + "_" + str(tau)
printR("setting: {}".format(setting))

# N20 tau1

In [ ]:
####################################
FQI_paras = {"hiddens" : [64, 64], "gamma" : gamma}
pi1 = FQI.FQI(gpu_number = 0, **FQI_paras)
pi1.model.load_weights(tp_path)
pi_behva = my_gym.softmax_policy(pi1, tau)
####################################
ohio = Ohio.OhioSimulator(sd_G = sd_G, T = T, N = N, behav = pi_behva)
V_true = ohio.eval_policy(pi = pi1, gamma = gamma, N = 10000)
init_S = ohio.simu_init_S(seed = 0, N = 1000)
ohio.eval_policy(pi = pi_behva, gamma = gamma, N = 1000)
######################################################################################## 
trajs_train_resp = Ohio.OhioSimulator(sd_G = sd_G, T = T, N = N * 500, behav = pi_behva).simu_one_seed(10000)
trajs_train_resp = [trajs_train_resp[(i * N):((i + 1) * N)] for i in range(rep)]


In [ ]:
rec = analyze.recorder()
rec.hyper = hyperparameters
for i in range(rep // n_gpu):
    ###########
    @ray.remote(num_gpus=1)
    def one_seed1(seed):
        gpu_idx = seed % n_gpu
        pi1 = FQI.FQI(gpu_number = gpu_idx, **FQI_paras)
        pi1.model.load_weights(tp_path)
        trajs_train = trajs_train_resp[seed]
        ############################ TRIPLE to estimate values ############################
        are = TRIPLE.ARE(trajs_train, pi = pi1, gamma = gamma, gpu_number = gpu_idx, incomplete_ratio = incomplete_ratio, L = 2)
        are.init_S = init_S 
        are.est_Q(**FQE_paras)
#         are.load_Q(Q_values = Qs[seed])
        are.est_w(h_dims = 512, batch_size = 256, lr = 0.0001, print_freq = 30, tolerance = 10, max_iter = 500, rep_loss = 5)
        are.est_IS()
        are.est_double_robust()
        #######
        are.est_cond_w(h_dims = 512, batch_size = 32, lr = 0.00005, print_freq = 30, tolerance = 10, max_iter = 500, rep_loss = 5)
        are.est_triply_robust()  
        are.est_quad_robust()
        #######
        printR("<<=== seed {} DONE! ===>>".format(seed))
        analyze.recorder().print_one_seed(V_true, are)
        return [are.raw_Qs, are.DR_V, are.TR_V, are.QR_V]
    ###########
    ray.init()
    futures = [one_seed1.remote(i) for i in range(n_gpu * i, n_gpu * (i + 1))]
    res = ray.get(futures)
    for j in range(n_gpu):
        rec.update(V_true, are_details = res[j])
    rec.analyze()
    rec.save("res/" + setting)
    ray.shutdown()